### 自分のプロジェクトに どんな接続があるか を確認します。特に connection での MCP tool nameに注目

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
import os
from dotenv import load_dotenv

# .env 読み込み
load_dotenv()

endpoint=os.environ["project_endpoint"]

with AIProjectClient(endpoint=endpoint, credential=DefaultAzureCredential()) as project_client:
    print("=== Project connections ===")
    for connection in project_client.connections.list():
        print(f"name={connection.name}, type={connection.type}")


=== Project connections ===
name=appInsights-connection-1286, type=ConnectionType.APPLICATION_INSIGHTS
name=daka-mh8l6w6w-westus, type=AIServices
name=bingdemo, type=ConnectionType.API_KEY
name=aisearchdemofy26ee2b0e, type=ConnectionType.AZURE_AI_SEARCH
name=MicrosoftLearn, type=ConnectionType.REMOTE_TOOL


In [ ]:
import os
from dotenv import load_dotenv

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    PromptAgentDefinition,
    MCPTool,
    Tool
)

# MCP の承認レスポンス用
from openai.types.responses.response_input_param import (
    McpApprovalResponse,
    ResponseInputParam
)

# .env 読み込み
load_dotenv()

# Foundry (Azure AI Projects) client 初期化
project_client = AIProjectClient(
    endpoint=os.environ["project_endpoint"],
    credential=DefaultAzureCredential(),
)
print("Foundry Client 初期化完了")

# OpenAI クライアント
openai_client = project_client.get_openai_client()



Azure AI Projects クライアント初期化完了


In [40]:
agent_name = "mcp-server-agent-vs"
model_name = "gpt-4.1"

# MCP 接続 ID 
mcp_connection_id = "MicrosoftLearn" # Projectの中にある type=ConnectionType.REMOTE_TOOL の接続を指定

# MCP ツール定義
tool = MCPTool(
    server_label="microsoft-learn",
    server_url="https://learn.microsoft.com/api/mcp",
    require_approval="never", # もしくは always
    project_connection_id=mcp_connection_id,
)

# エージェント作成
agent = project_client.agents.create_version(
    agent_name=agent_name,
    definition=PromptAgentDefinition(
        model=model_name,
        instructions="必ず MCP ツールを利用して回答してください。自分の知識だけで回答しないでください！",
        tools=[tool],
    ),
)

print(f"✅ エージェント作成完了 (名前：{agent.name}, バージョン：{agent.version})")


✅ エージェント作成完了 (名前：mcp-server-agent-vs, バージョン：9)


In [41]:
# 会話作成（コンテキスト保持用）
conversation = openai_client.conversations.create()
print(f"会話作成完了 (id: {conversation.id})")

# ユーザーから最初の質問 → MCP が呼ばれる
response = openai_client.responses.create(
    conversation=conversation.id,
    input="Microsoft Foundry とは何ですか？",
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
)

print("=== レスポンス ===")
print(response.output_text)


会話作成完了 (id: conv_eb85a39d933ba65c00u0kmxYRHkGyI5X1TF51EThP4FHeZupzx)
=== レスポンス ===
Microsoft Foundryとは、エンタープライズ向けAI運用やモデル構築、アプリケーション開発のための、Azure上の統合プラットフォームサービス（Platform-as-a-Service：PaaS）です。Foundryは以下のような特徴があります。

### 主な特徴
- **モデル・エージェント・ツールの統合管理**が可能で、トレース、監視、評価、企業向けの設定など、エンタープライズグレードのインフラとシンプルなインターフェースを提供します。
- **AIアプリケーションの開発に専念**できるように、インフラの管理負担を大幅に軽減します。
- **一元化されたRBAC（ロールベースアクセス制御）やネットワーク、ポリシー管理**により、セキュアでスケーラブルなAIプロジェクト運用を支援します。
- **様々なAIモデルの探索、構築、テスト、デプロイが容易**であり、責任あるAI実践にも対応。
- **Foundryプロジェクト**単位で開発・隔離環境を簡単に作成でき、チームでの共同作業やデータの分離が可能です。
- **Microsoft Foundry APIとSDK（Python, C#, JavaScript/TypeScript, Java）**によって、さまざまな言語・環境でAI機能を組み込めます。

### 提供される主な2種類のポータル
1. **Microsoft Foundry (classic)**  
   複数のリソースタイプ（Azure OpenAI, Foundryリソース, ハブベースプロジェクト等）を扱う場合に使用。
2. **Microsoft Foundry (new)**  
   よりシンプルかつ強力なマルチエージェントアプリケーション開発のための新しいポータル。

### 参考ドキュメント
より詳細な説明や最新情報は、以下の公式ドキュメントを参照してください。
- [What is Microsoft Foundry?（公式ドキュメント）](https://learn.microsoft.com/en-us/azure/ai-foundry/what